### Imports and Setup

In [1]:
import numpy as np
import random
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# sklearn
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF, WhiteKernel,ExpSineSquared,DotProduct,RationalQuadratic
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# scipy
from scipy import sparse

# custom packages
from packages.gridsearch import gridsearch as gs



### Set random seed
seed = 5
random.seed(seed)
np.random.seed(seed)

### Suppresses Warning
import warnings
warnings.filterwarnings('ignore')

# 1. Data Prep

### 1.1. Load

In [2]:
data = pd.read_csv('data/hw3_data.csv', delimiter=',',header=0)
print(data.shape)

(9051, 4)


### 1.2. Encode

In [3]:
# create separate columns for each amino acid
for i in range(9): # all seq are length 9
    colname='seq'+str(i)
    data[colname] = [x[i] for x in data['seq']]

# separate features and target, remove unnecessary columns
X_df = data.drop(['pIC50','id','allele', 'seq'],axis=1)
y = data['pIC50']

# encode features
enc = OneHotEncoder(handle_unknown='ignore')
X_enc = enc.fit_transform(X_df)

# ?? standardize target

# convert to numpy array ?? is this necessary
X_pool=sparse.csr_matrix.toarray(X_enc)
y_pool = y.to_numpy()


### 1.3. Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_pool, y_pool, test_size=0.33)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(6064, 180)
(6064,)
(2987, 180)
(2987,)


# 2. Experiments with offline learners and kernels
To see if a particular regressor or kernel works better on the data than any other.

### 2.1 Random Forest Regressor
Doesn't meet 0.6 threshold.

In [5]:
# %%time

# # train RFC model on entire pool of data
# rf = RandomForestRegressor(n_estimators = 20, 
#                             max_depth = 6, 
#                             random_state = seed)
# rf.fit(X_train, y_train)

# # calculate accuracy
# print(rf.score(X_test,y_test)) #0.476

### 2.2 Ridge Regression
Barely passable to meet 0.6 threshold.

In [6]:
# %%time

# clf = Ridge(alpha=1.0)
# clf.fit(X_train, y_train)

# print(clf.score(X_test,y_test))  #0.632

### 2.3 Ridge Regression with variable alpha
Did not dramatically improve score. (0.63 -> 0.65 maybe)

In [7]:
# # grid search had no significant improvement 
# for i in np.linspace(0.1,5,50):
#     clf = Ridge(alpha=i)
#     clf.fit(X_train, y_train)

#     print(np.round(i,2),np.round(clf.score(X_test,y_test),3))

### 2.4 RBF Kernel
Good score.

In [8]:
# %%time
# #2 min

# # checking if WhiteKernel is helping or not
# kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3))

# gpr = GaussianProcessRegressor(kernel,random_state=seed)
# gpr.fit(X_train, y_train)
# print(gpr.score(X_test,y_test)) #0.6885

### 2.5 RBF + WhiteKernel
WhiteKernel doesn't seem to improve score. Good score.

In [9]:
# %%time
# # 15 minutes for (5931,180)

# check if WhiteKernel helps or not
# kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3)) \
#          + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

# gpr = GaussianProcessRegressor(kernel,random_state=seed)
# gpr.fit(X_train, y_train)
# print(gpr.score(X_test,y_test)) #0.686 score

### 2.6 RBF no length_scale_bounds
Seems to increase runtime slightly. No effect on results. Good score.

In [10]:
# %%time
# # 2 min

# # checking if bounds is helping or not
# kernel = RBF(length_scale=1.0)

# gpr = GaussianProcessRegressor(kernel,random_state=seed)
# gpr.fit(X_train, y_train)
# print(gpr.score(X_test,y_test))  #0.6885

### 2.7 RationalQuadratic kernel
Good score

In [11]:
# %%time
# # 3.5 min

# kernel = RationalQuadratic(length_scale=1.0, alpha=1.5, length_scale_bounds=(1e-2, 1e3))
# gpr = GaussianProcessRegressor(kernel,random_state=seed)
# gpr.fit(X_train, y_train)
# print(gpr.score(X_test,y_test)) #0.6888

### 2.8 DotProduct + WhiteKernel
Barely passable score.

In [12]:
# %%time
# # 2 min

# kernel = DotProduct() + WhiteKernel()
# gpr = GaussianProcessRegressor(kernel,random_state=seed)
# gpr.fit(X_train, y_train)
# print(gpr.score(X_test,y_test)) # 0.632

### 2.9 Default Kernel
Bad.

In [13]:
# %%time
# # 15 s

# gpr = GaussianProcessRegressor(random_state=seed)
# gpr.fit(X_train, y_train)
# print(gpr.score(X_test,y_test)) # -0.806

### 2.10 RBF + ExpSineSquared
Idea is that sequences are "periodic" data, and sequences repeat in groups. If we combine RBF and periodic kernel we could better model the data. Model wouldn't run due to LinAlgError:

```
LinAlgError: ("The kernel, RBF(length_scale=1) + ExpSineSquared(length_scale=1, periodicity=1), is not returning a positive definite matrix. Try gradually increasing the 'alpha' parameter of your GaussianProcessRegressor estimator.", '10-th leading minor of the array is not positive definite')
```

Attempted to modify alpha to fix issue, to no avail.

In [14]:
# %%time

# from numpy.linalg import LinAlgError

# kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3)) + ExpSineSquared(length_scale=1, periodicity=1)

# for alpha in [1E-9,1E-8,1E-7,1E-6,1E-5,1E-4,1E-3,1E-2,1E-1,1E-0]:
        
# #     try:
# #         gpr = GaussianProcessRegressor(kernel,random_state=seed, alpha=alpha)
# #         gpr.fit(X_train, y_train)
# #         print(alpha, gpr.score(X_test,y_test))  
# #     except LinAlgError:
# #         print(alpha, "Error")


### 2.11 Matern
Good model.

In [15]:
# %%time
# # 2 min

# kernel = Matern(length_scale=1.0, nu=1.5)

# gpr = GaussianProcessRegressor(kernel,random_state=seed)
# gpr.fit(X_train, y_train)
# print(gpr.score(X_test,y_test)) # 0.687

# 3. Active Learning model - Grid Search v1
Experiments to test out various combinations to achieve $R^2 \ge 0.6$ threshold.

All experiments in this section use the following configurations:
- Category: Committee
- Learner: Gaussian Process
- query_strategy: max_std_sampling

In [16]:
#?? Cookbook suggests "put a product of SE kernels on those dimensions" (I have 180 dimensions)

### Functions

In [17]:
def matern_details(kernel):
    return [kernel,kernel.length_scale, kernel.nu]

In [18]:
def rbf_details(kernel):
    return [kernel, kernel.length_scale, kernel.length_scale_bounds]

In [19]:
def rationalquadratic_details(kernel):
    return [kernel, kernel.length_scale, kernel.alpha]

In [20]:
def rbf_plus_white(kernel):
    return [kernel, kernel.k1.length_scale, kernel.k2.noise_level]

### 3.1. Matern
- 26 minutes with `2*2*10` Matern configurations at 100 queries - 2,5 learners
- 3 hours 4 minutes with `2*2*10` Matern configurations at 100 queries - 10,20 learners


In [21]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','length_scale', 'nu', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [Matern(length_scale=i, nu=1.5) for i in np.linspace(0.5,1,2)]

# n_learners = [2]
# n_initials = [10]
# n_queries = 1

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, matern_details)


### 3.2. RBF

In [22]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.linspace(0.38,0.418,2)]

# n_learners = [2]
# n_initials = [10]
# n_queries = 1

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


### 3.3. RationalQuadratic

In [23]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','length_scale', 'alpha', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RationalQuadratic(length_scale=i, alpha=j, length_scale_bounds=(1e-2, 1e3)) 
#            for i in np.linspace(0.5,10,2) 
#            for j in np.linspace(0.5,2,2)]

# n_learners = [2]
# n_initials = [10]
# n_queries = 1

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rationalquadratic_details)

### 3.4. RBF + WhiteKernel random search
Stopped halfway through because results weren't interesting. Adding noise doesn't improve score.

In [24]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','rbf_length_scale', 'white_noise_scale', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=0.45,length_scale_bounds=(1e-2, 1e3)) \
#            + WhiteKernel(noise_level=i, noise_level_bounds=(1e-10, 1e+1)) 
#            for i in np.random.uniform(low=0.1,high=1.0,size=20)]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_plus_white)

### 3.5. RBF linear search wide
Search for peaks.

In [25]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i) for i in [1e-10,1e-5,1e-2,1e-1,1,1e1,100]]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


### 3.6. RBF linear search narrow
Search near current peaks.

In [26]:
# %%time
# # 1 hour

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i) for i in np.linspace(0.1,1,10)] \
#         + [RBF(length_scale=i) for i in np.linspace(1,10,10)]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


### 3.7. RBF linear search narrow 2
Search near current peaks. Skipped fifth run due to time.

In [27]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i) for i in np.linspace(0.4,7,40)] \
#         + [RBF(length_scale=i) for i in np.linspace(0.4,7,40)] \
#         + [RBF(length_scale=i) for i in np.linspace(0.4,7,40)] \
#         + [RBF(length_scale=i) for i in np.linspace(0.4,7,40)] \
#         + [RBF(length_scale=i) for i in np.linspace(0.4,7,40)]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


### 3.8. RBF linear search narrow 3
Search near current peaks.

In [28]:
# %%time
# # 3h 4 min

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'
# row_prefix = ['Committee','Gaussian Process','max_std_sampling']
# fields = ['category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.linspace(0.45,0.55,10)] \
#         + [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.linspace(0.7,0.8,10)] \
#         + [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.linspace(2.4,3.0,10)] \
#         + [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.linspace(3.7,4.0,10)] \
#         + [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.linspace(4.8,5.2,10)]  

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gaussian_process_regressor_gs(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


### 3.9. RBF random search narrow
Randomized points near peaks.

In [29]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'

# ex_id = ['3.9']
# row_prefix = ex_id + ['Committee','Gaussian Process','max_std_sampling']
# fields = ['experiment','category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.random.uniform(low=2.4,high=3.0,size=20)] \
#         + [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.random.uniform(low=3.8,high=4.0,size=20)] \
#         + [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.random.uniform(low=4.8,high=5.0,size=20)] \
#         + [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.random.uniform(low=0.49,high=0.51,size=20)]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gs.grid_search_1(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


### 3.10. RBF best points
Checking rounded values near the highest R2 points.

In [30]:
# %%time


# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'

# ex_id = ['3_10']
# row_prefix = ex_id + ['Committee','Gaussian Process','max_std_sampling']
# fields = ['experiment','category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3))
#            for i in [0.500,0.738,1.923,3.785,5.985,2.769,2.760,2.412,4.983]]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gs.grid_search_1(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


### 3.11. Matern wide search - scale_length
Checking range of Matern values

In [31]:
# %%time


# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'

# ex_id = ['3_11']
# row_prefix = ex_id + ['Committee','Gaussian Process','max_std_sampling']
# fields = ['experiment','category','learner','query_strategy','kernel','length_scale', 'nu', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = kernels = [Matern(length_scale=i, nu=1.5) for i in [1e-10,1e-5,1e-2,1e-1,1,1e1,100]]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gs.grid_search_1(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, matern_details)


### 3.12. RBF Random Search ?
Add additional points to clarify score shape.

In [32]:
# %%time


# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'

# ex_id = ['3_10']
# row_prefix = ex_id + ['Committee','Gaussian Process','max_std_sampling']
# fields = ['experiment','category','learner','query_strategy','kernel','length_scale', 'length_scale_bounds', 'n_learners', 'n_initial','n_queries','r2']

# # configs
# kernels = [RBF(length_scale=i,length_scale_bounds=(1e-2, 1e3)) for i in np.random.uniform(low=0.4,high=5.1,size=200)]

# n_learners = [10]
# n_initials = [80]
# n_queries = 100

# # run process
# gs.grid_search_1(kernels,n_learners,n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, rbf_details)


# 4. Active learning model - Grid Search v2
Experiments to test out various combinations to achieve highest starting $R^2$ value.

All experiments in this section use the following configurations:
- Category: Single Learner
- Learner: Random Forest Regressor
- query_strategy: GP_regression_std

### 4.1 Grid Search over hyperparameters
Find a good starting configuration for Random Forest regression

In [33]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'

# ex_id = ['4_1']
# row_prefix = ex_id + ['Single Learner','Random Forest Regressor','GP_regression_std']
# fields = ['experiment','category','learner','query_strategy','n_estimators','max_depth','n_learners', 'n_initial','n_queries','r2']


# # configs
# max_depths = [i for i in range(1,20)]
# n_estimators = [i for i in range(1,100,5)]
# n_learners = [10]
# n_initials = [80]
# n_queries = 1

# # run process
# gs.grid_search_2(n_estimators, max_depths, n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, randomforestdetails)


### 4.2 Grid Search over hyperparameters
Find a good starting configuration for Random Forest regression

In [34]:
# %%time

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'

# ex_id = ['4_2']
# row_prefix = ex_id + ['Single Learner','Random Forest Regressor','GP_regression_std']
# fields = ['experiment','category','learner','query_strategy','n_estimators','max_depth','n_learners', 'n_initial','n_queries','r2']


# # configs
# max_depths = [i for i in range(1,20)]
# n_estimators = [i for i in range(100,500,25)]
# n_learners = [10]
# n_initials = [80]
# n_queries = 1

# # run process
# gs.grid_search_2(n_estimators, max_depths, n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix, randomforestdetails)


### 4.3 Grid Search over hyperparameters
Repeat to reduce variance in results.

In [35]:
# %%time
# 22min

# # results file
# filename = 'data/gridsearch.' + datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S') + '.csv'

# ex_id = ['4_3']
# row_prefix = ex_id + ['Single Learner','Random Forest Regressor','GP_regression_std']
# fields = ['experiment','category','learner','query_strategy','n_estimators','max_depth','n_learners', 'n_initial','n_queries','r2']


# # configs
# max_depths =  [i for i in range(1,20)] \
#             + [i for i in range(1,20)] \
#             + [i for i in range(1,20)] \
#             + [i for i in range(1,20)]

# n_estimators = [i for i in range(1,100,5)] \
#              + [i for i in range(1,100,5)] \
#              + [i for i in range(1,100,5)] \
#              + [i for i in range(1,100,5)]

# n_learners = [10]
# n_initials = [80]
# n_queries = 1

# # run process
# gs.grid_search_2(n_estimators, max_depths, n_initials,X_train,y_train,X_test,y_test,n_queries,
#                                         seed, filename, fields, row_prefix)


RandomForestRegressor(n_estimators=1,max_depth=1)|80|1|0.10142437539500837
RandomForestRegressor(n_estimators=1,max_depth=2)|80|1|-0.04377035969021148
RandomForestRegressor(n_estimators=1,max_depth=3)|80|1|-0.06232390243593211
RandomForestRegressor(n_estimators=1,max_depth=4)|80|1|-0.003967791741799509
RandomForestRegressor(n_estimators=1,max_depth=5)|80|1|-0.13568445244677796
RandomForestRegressor(n_estimators=1,max_depth=6)|80|1|-0.4847508583906277
RandomForestRegressor(n_estimators=1,max_depth=7)|80|1|-0.17150960428949302
RandomForestRegressor(n_estimators=1,max_depth=8)|80|1|-0.07822177867356661
RandomForestRegressor(n_estimators=1,max_depth=9)|80|1|-0.5861810121164188
RandomForestRegressor(n_estimators=1,max_depth=10)|80|1|-0.11162077587822661
RandomForestRegressor(n_estimators=1,max_depth=11)|80|1|-0.498956244240216
RandomForestRegressor(n_estimators=1,max_depth=12)|80|1|-0.40127210859407825
RandomForestRegressor(n_estimators=1,max_depth=13)|80|1|-0.39423309185127575
RandomForest

RandomForestRegressor(n_estimators=6,max_depth=18)|80|1|0.03715347995160356
RandomForestRegressor(n_estimators=6,max_depth=19)|80|1|0.0668002860451321
RandomForestRegressor(n_estimators=6,max_depth=1)|80|1|0.22644093220096484
RandomForestRegressor(n_estimators=6,max_depth=2)|80|1|0.160188060193857
RandomForestRegressor(n_estimators=6,max_depth=3)|80|1|0.29278190443549745
RandomForestRegressor(n_estimators=6,max_depth=4)|80|1|0.12870422982917296
RandomForestRegressor(n_estimators=6,max_depth=5)|80|1|0.14819884896844293
RandomForestRegressor(n_estimators=6,max_depth=6)|80|1|0.26781276229530515
RandomForestRegressor(n_estimators=6,max_depth=7)|80|1|0.25387750902671646
RandomForestRegressor(n_estimators=6,max_depth=8)|80|1|0.21694116583267653
RandomForestRegressor(n_estimators=6,max_depth=9)|80|1|0.06310171425319278
RandomForestRegressor(n_estimators=6,max_depth=10)|80|1|0.2087166164333527
RandomForestRegressor(n_estimators=6,max_depth=11)|80|1|0.1026382446723294
RandomForestRegressor(n_es

RandomForestRegressor(n_estimators=11,max_depth=16)|80|1|0.2451907475091203
RandomForestRegressor(n_estimators=11,max_depth=17)|80|1|0.2895448083822889
RandomForestRegressor(n_estimators=11,max_depth=18)|80|1|0.2651997898950962
RandomForestRegressor(n_estimators=11,max_depth=19)|80|1|0.2695374139635124
RandomForestRegressor(n_estimators=16,max_depth=1)|80|1|0.11875663439923156
RandomForestRegressor(n_estimators=16,max_depth=2)|80|1|0.2798111950192387
RandomForestRegressor(n_estimators=16,max_depth=3)|80|1|0.18009286855222528
RandomForestRegressor(n_estimators=16,max_depth=4)|80|1|0.11521080539078377
RandomForestRegressor(n_estimators=16,max_depth=5)|80|1|0.20490174419746865
RandomForestRegressor(n_estimators=16,max_depth=6)|80|1|0.18589646192701903
RandomForestRegressor(n_estimators=16,max_depth=7)|80|1|0.1582809520700562
RandomForestRegressor(n_estimators=16,max_depth=8)|80|1|0.3034125736639056
RandomForestRegressor(n_estimators=16,max_depth=9)|80|1|0.21379435783606227
RandomForestReg

RandomForestRegressor(n_estimators=21,max_depth=11)|80|1|0.277659390628303
RandomForestRegressor(n_estimators=21,max_depth=12)|80|1|0.3425476641266787
RandomForestRegressor(n_estimators=21,max_depth=13)|80|1|0.2241728773177154
RandomForestRegressor(n_estimators=21,max_depth=14)|80|1|0.17465883080763378
RandomForestRegressor(n_estimators=21,max_depth=15)|80|1|0.26313789720666236
RandomForestRegressor(n_estimators=21,max_depth=16)|80|1|0.20786785468187008
RandomForestRegressor(n_estimators=21,max_depth=17)|80|1|0.17095861847791316
RandomForestRegressor(n_estimators=21,max_depth=18)|80|1|0.2394650922021958
RandomForestRegressor(n_estimators=21,max_depth=19)|80|1|0.1351835465410327
RandomForestRegressor(n_estimators=21,max_depth=1)|80|1|0.18256976244624057
RandomForestRegressor(n_estimators=21,max_depth=2)|80|1|0.2458616583962051
RandomForestRegressor(n_estimators=21,max_depth=3)|80|1|0.22373124465711913
RandomForestRegressor(n_estimators=21,max_depth=4)|80|1|0.21333402444598248
RandomFore

RandomForestRegressor(n_estimators=26,max_depth=5)|80|1|0.2881344326882973
RandomForestRegressor(n_estimators=26,max_depth=6)|80|1|0.2609196989634015
RandomForestRegressor(n_estimators=26,max_depth=7)|80|1|0.20933885743317593
RandomForestRegressor(n_estimators=26,max_depth=8)|80|1|0.2540260372480061
RandomForestRegressor(n_estimators=26,max_depth=9)|80|1|0.22236972502658925
RandomForestRegressor(n_estimators=26,max_depth=10)|80|1|0.22019947934102235
RandomForestRegressor(n_estimators=26,max_depth=11)|80|1|0.27104106205448053
RandomForestRegressor(n_estimators=26,max_depth=12)|80|1|0.23121276314277783
RandomForestRegressor(n_estimators=26,max_depth=13)|80|1|0.3092508096333181
RandomForestRegressor(n_estimators=26,max_depth=14)|80|1|0.33163867600192987
RandomForestRegressor(n_estimators=26,max_depth=15)|80|1|0.28083755063905025
RandomForestRegressor(n_estimators=26,max_depth=16)|80|1|0.23200146717668535
RandomForestRegressor(n_estimators=26,max_depth=17)|80|1|0.3111692381510516
RandomFor

...

RandomForestRegressor(n_estimators=66,max_depth=11)|80|1|0.24786205180082577
RandomForestRegressor(n_estimators=66,max_depth=12)|80|1|0.3419820579739248
RandomForestRegressor(n_estimators=66,max_depth=13)|80|1|0.31641686101561983
RandomForestRegressor(n_estimators=66,max_depth=14)|80|1|0.34788771602905333
RandomForestRegressor(n_estimators=66,max_depth=15)|80|1|0.2927135825752165
RandomForestRegressor(n_estimators=66,max_depth=16)|80|1|0.22681495569651067
RandomForestRegressor(n_estimators=66,max_depth=17)|80|1|0.24826544273644358
RandomForestRegressor(n_estimators=66,max_depth=18)|80|1|0.0841981687873199
RandomForestRegressor(n_estimators=66,max_depth=19)|80|1|0.16728903456619293
RandomForestRegressor(n_estimators=66,max_depth=1)|80|1|0.13741765239145975
RandomForestRegressor(n_estimators=66,max_depth=2)|80|1|0.16117741882691383
RandomForestRegressor(n_estimators=66,max_depth=3)|80|1|0.291091999516061
RandomForestRegressor(n_estimators=66,max_depth=4)|80|1|0.32386666630214833
RandomFo

RandomForestRegressor(n_estimators=71,max_depth=5)|80|1|0.186932290983425
RandomForestRegressor(n_estimators=71,max_depth=6)|80|1|0.3491098942514995
RandomForestRegressor(n_estimators=71,max_depth=7)|80|1|0.20664075892845168
RandomForestRegressor(n_estimators=71,max_depth=8)|80|1|0.23939823893215328
RandomForestRegressor(n_estimators=71,max_depth=9)|80|1|0.33979092313526105
RandomForestRegressor(n_estimators=71,max_depth=10)|80|1|0.2631124621990856
RandomForestRegressor(n_estimators=71,max_depth=11)|80|1|0.25129950744315077
RandomForestRegressor(n_estimators=71,max_depth=12)|80|1|0.2773572561018648
RandomForestRegressor(n_estimators=71,max_depth=13)|80|1|0.28547485929503125
RandomForestRegressor(n_estimators=71,max_depth=14)|80|1|0.30834338935457306
RandomForestRegressor(n_estimators=71,max_depth=15)|80|1|0.21677735263963105
RandomForestRegressor(n_estimators=71,max_depth=16)|80|1|0.19830594870169782
RandomForestRegressor(n_estimators=71,max_depth=17)|80|1|0.1979315129583782
RandomFore

RandomForestRegressor(n_estimators=76,max_depth=18)|80|1|0.2805971266879159
RandomForestRegressor(n_estimators=76,max_depth=19)|80|1|0.3205462827326886
RandomForestRegressor(n_estimators=81,max_depth=1)|80|1|0.21288088161037888
RandomForestRegressor(n_estimators=81,max_depth=2)|80|1|0.19330650464683252
RandomForestRegressor(n_estimators=81,max_depth=3)|80|1|0.26062633157780923
RandomForestRegressor(n_estimators=81,max_depth=4)|80|1|0.28408719620627254
RandomForestRegressor(n_estimators=81,max_depth=5)|80|1|0.29417511158725496
RandomForestRegressor(n_estimators=81,max_depth=6)|80|1|0.2609689469861921
RandomForestRegressor(n_estimators=81,max_depth=7)|80|1|0.16973332149518028
RandomForestRegressor(n_estimators=81,max_depth=8)|80|1|0.2718926646466733
RandomForestRegressor(n_estimators=81,max_depth=9)|80|1|0.16270106678913654
RandomForestRegressor(n_estimators=81,max_depth=10)|80|1|0.2709370565240504
RandomForestRegressor(n_estimators=81,max_depth=11)|80|1|0.280991544209493
RandomForestReg

RandomForestRegressor(n_estimators=86,max_depth=12)|80|1|0.30264855624609144
RandomForestRegressor(n_estimators=86,max_depth=13)|80|1|0.2634100107279993
RandomForestRegressor(n_estimators=86,max_depth=14)|80|1|0.2600579074480047
RandomForestRegressor(n_estimators=86,max_depth=15)|80|1|0.261891493553963
RandomForestRegressor(n_estimators=86,max_depth=16)|80|1|0.3353330985511863
RandomForestRegressor(n_estimators=86,max_depth=17)|80|1|0.20174778121532766
RandomForestRegressor(n_estimators=86,max_depth=18)|80|1|0.11349176200817024
RandomForestRegressor(n_estimators=86,max_depth=19)|80|1|0.26036374379542515
RandomForestRegressor(n_estimators=86,max_depth=1)|80|1|0.19626968008304724
RandomForestRegressor(n_estimators=86,max_depth=2)|80|1|0.26357552818267593
RandomForestRegressor(n_estimators=86,max_depth=3)|80|1|0.2610666798900382
RandomForestRegressor(n_estimators=86,max_depth=4)|80|1|0.1685882007718521
RandomForestRegressor(n_estimators=86,max_depth=5)|80|1|0.2639354419793567
RandomForest

RandomForestRegressor(n_estimators=91,max_depth=6)|80|1|0.23551831056111427
RandomForestRegressor(n_estimators=91,max_depth=7)|80|1|0.2724942147958729
RandomForestRegressor(n_estimators=91,max_depth=8)|80|1|0.25531885424091094
RandomForestRegressor(n_estimators=91,max_depth=9)|80|1|0.19912606703466473
RandomForestRegressor(n_estimators=91,max_depth=10)|80|1|0.23924118760163782
RandomForestRegressor(n_estimators=91,max_depth=11)|80|1|0.19689247238479346
RandomForestRegressor(n_estimators=91,max_depth=12)|80|1|0.29173816765264793
RandomForestRegressor(n_estimators=91,max_depth=13)|80|1|0.18621062364225838
RandomForestRegressor(n_estimators=91,max_depth=14)|80|1|0.24871035986141565
RandomForestRegressor(n_estimators=91,max_depth=15)|80|1|0.33103323144677355
RandomForestRegressor(n_estimators=91,max_depth=16)|80|1|0.2277070484100212
RandomForestRegressor(n_estimators=91,max_depth=17)|80|1|0.32385546604125204
RandomForestRegressor(n_estimators=91,max_depth=18)|80|1|0.3020454928785983
Random